# Duplicate Labels

`Index` objects are not required to be unique; you can have duplicate row
or column labels. This may be a bit confusing at first. If you’re familiar with
SQL, you know that row labels are similar to a primary key on a table, and you
would never want duplicates in a SQL table. But one of pandas’ roles is to clean
messy, real-world data before it goes to some downstream system. And real-world
data has duplicates, even in fields that are supposed to be unique.

This section describes how duplicate labels change the behavior of certain
operations, and how prevent duplicates from arising during operations, or to
detect them if they do.

In [20]:
import pandas as pd
import numpy as np

## Consequences of Duplicate Labels

Some pandas methods (`Series.reindex()` for example) just don’t work with
duplicates present. The output can’t be determined, and so pandas raises an exception.

```python

s1 = pd.Series([0, 1, 2], index=["a", "b", "b"])
s1.reindex(["a", "b", "c"])
#ValueError: cannot reindex from a duplicate axis
``` 

Other methods, like indexing, can give very surprising results. Typically
indexing with a scalar will *reduce dimensionality*. Slicing a `DataFrame`
with a scalar will return a `Series`. Slicing a `Series` with a scalar will
return a scalar. But with duplicates, this isn’t the case.

In [21]:
df1 = pd.DataFrame([[0, 1, 2], [3, 4, 5]], columns=["A", "A", "B"])
df1

,A,A,B
0,0,1,2
1,3,4,5


We have duplicates in the columns. If we slice `'B'`, we get back a `Series`

In [22]:
df1["B"]  # a series

0    2
1    5
Name: B, dtype: int64

But slicing `'A'` returns a `DataFrame`

In [23]:
df1["A"]  # a DataFrame

,A,A
0,0,1
1,3,4


This applies to row labels as well

In [24]:
df2 = pd.DataFrame({"A": [0, 1, 2]}, index=["a", "a", "b"])
df2

,A
a,0
a,1
b,2


In [25]:
df2.loc["b", "A"]  # a scalar

2

In [26]:
df2.loc["a", "A"]  # a Series

a    0
a    1
Name: A, dtype: int64

## Duplicate Label Detection

You can check whether an `Index` (storing the row or column labels) is
unique with `Index.is_unique`:

In [27]:
df2

,A
a,0
a,1
b,2


In [28]:
df2.index.is_unique

False

In [29]:
df2.columns.is_unique

True

>**Note**
>
>Checking whether an index is unique is somewhat expensive for large datasets.
pandas does cache this result, so re-checking on the same index is very fast.

`Index.duplicated()` will return a boolean ndarray indicating whether a
label is repeated.

In [30]:
df2.index.duplicated()

array([False,  True, False])

Which can be used as a boolean filter to drop duplicate rows.

In [31]:
df2.loc[~df2.index.duplicated(), :]

,A
a,0
b,2


If you need additional logic to handle duplicate labels, rather than just
dropping the repeats, using `groupby()` on the index is a common
trick. For example, we’ll resolve duplicates by taking the average of all rows
with the same label.

In [32]:
df2.groupby(level=0).mean()

,A
a,0.5
b,2.0



<a id='duplicates-disallow'></a>

## Disallowing Duplicate Labels

New in version 1.2.0.

As noted above, handling duplicates is an important feature when reading in raw
data. That said, you may want to avoid introducing duplicates as part of a data
processing pipeline (from methods like `pandas.concat()`,
`rename()`, etc.). Both `Series` and `DataFrame`
*disallow* duplicate labels by calling `.set_flags(allows_duplicate_labels=False)`.
(the default is to allow them). If there are duplicate labels, an exception
will be raised.

```Python

pd.Series([0, 1, 2], index=["a", "b", "b"]).set_flags(allows_duplicate_labels=False)
DuplicateLabelError: Index has duplicates.
```

In [33]:
pd.Series([0, 1, 2], index=["a", "b", "b"]).set_flags(allows_duplicate_labels=True)

a    0
b    1
b    2
dtype: int64

This applies to both row and column labels for a `DataFrame`

In [34]:
pd.DataFrame([[0, 1, 2], [3, 4, 5]], columns=["A", "B", "C"],).set_flags(
    allows_duplicate_labels=False
)

,A,B,C
0,0,1,2
1,3,4,5


This attribute can be checked or set with `allows_duplicate_labels`,
which indicates whether that object can have duplicate labels.

In [35]:
df = pd.DataFrame({"A": [0, 1, 2, 3]}, index=["x", "y", "X", "Y"]).set_flags(
    allows_duplicate_labels=False
)
df

,A
x,0
y,1
X,2
Y,3


In [36]:
df.flags.allows_duplicate_labels

False

`DataFrame.set_flags()` can be used to return a new `DataFrame` with attributes
like `allows_duplicate_labels` set to some value

In [37]:
df2 = df.set_flags(allows_duplicate_labels=True)

In [38]:
df2.flags.allows_duplicate_labels

True

The new `DataFrame` returned is a view on the same data as the old `DataFrame`.
Or the property can just be set directly on the same object

In [39]:
df2.flags.allows_duplicate_labels = False

In [40]:
df2.flags.allows_duplicate_labels

False

When processing raw, messy data you might initially read in the messy data
(which potentially has duplicate labels), deduplicate, and then disallow duplicates
going forward, to ensure that your data pipeline doesn’t introduce duplicates.

```python
>>> raw = pd.read_csv("...")
>>> deduplicated = raw.groupby(level=0).first()  # remove duplicates
>>> deduplicated.flags.allows_duplicate_labels = False  # disallow going forward
```


Setting `allows_duplicate_labels=True` on a `Series` or `DataFrame` with duplicate
labels or performing an operation that introduces duplicate labels on a `Series` or
`DataFrame` that disallows duplicates will raise an
`errors.DuplicateLabelError`.

``` Python

df.rename(str.upper)
DuplicateLabelError: Index has duplicates.
```

This error message contains the labels that are duplicated, and the numeric positions
of all the duplicates (including the “original”) in the `Series` or `DataFrame`

### Duplicate Label Propagation

In general, disallowing duplicates is “sticky”. It’s preserved through
operations.

In [ ]:
s1 = pd.Series(0, index=["a", "b"]).set_flags(allows_duplicate_labels=False)
s1

In [ ]:
s1.head().rename({"a": "b"})

This is an experimental feature. Currently, many methods fail to
propagate the `allows_duplicate_labels` value. In future versions
it is expected that every method taking or returning one or more
DataFrame or Series objects will propagate `allows_duplicate_labels`.